In [63]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
    raw_text = f.read()

print("Total no. of. charachters" , len(raw_text))
print(raw_text[:99])

Total no. of. charachters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [64]:
import re

text = "Hello world ! , This is a test"
result = re.split(r'(\s)' , text)

print(result)

['Hello', ' ', 'world', ' ', '!', ' ', ',', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test']


In [65]:
result = re.split(r'([,.;:?_!"()\']|--|\s)' , text)
print(result)

['Hello', ' ', 'world', ' ', '', '!', '', ' ', '', ',', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test']


In [66]:
## removing the white spaces
result = [item for item in result if item.strip()]
print(result)

['Hello', 'world', '!', ',', 'This', 'is', 'a', 'test']


In [67]:
preprocessed = re.split(r'([,.;:?_!"()\']|--|\s)' , raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [68]:
print(len(preprocessed))

4690


In [69]:
### sort all words in alfabetical order 
## set keyword is used to take only unique words (non-repeating)
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [70]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [71]:
## Let's print some of them to understand 

for i,item in enumerate(vocab.items()):
    print(item)
    if(i>=20):
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [72]:
### THE ABOVE PROCESS IS ENCODER 
### reversal of this process is called DECODER

In [73]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int = vocab  ##mentioned above list
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.;:?_!"()\']|--|\s)' , text)
        preprocessed = [ item.strip() for item in preprocessed if item.strip() ]
        ids = [ self.str_to_int[s] for s in preprocessed ]
        return ids

    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        ## Replace spaces before special charachters
        text = re.sub(r'\s+([,.?!"()\'])' , r'\1' , text)
        return text        

In [74]:
### Let's test it

tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted you know,"
        Mrs. Gisburn said with pardonable pride.""" 

In [75]:
### Test for encode function whether it convert these text into ids and returning it 
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [76]:
tokenizer.decode(ids)

'" It\' s the last he painted you know," Mrs. Gisburn said with pardonable pride.'

In [77]:
### What if we add text that is not in book ?

tokenizer = SimpleTokenizerV1(vocab)
text = "Let's have tea ?"

In [78]:
ids = tokenizer.encode(text)
print(ids)

KeyError: 'Let'

In [79]:
### See the above wont't work as these keywords are not in book
### So , Let's add some more tokens like unknown , end of text .

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [80]:
len(vocab)

1132

In [81]:
## Let's print last 5 words in vocab
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [82]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab  ##mentioned above list
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.;:?_!"()\']|--|\s)' , text)
        preprocessed = [ item.strip() for item in preprocessed if item.strip() ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [ self.str_to_int[s] for s in preprocessed ]
        return ids

    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        ## Replace spaces before special charachters
        text = re.sub(r'\s+([,.?!"()\'])' , r'\1' , text)
        return text        

In [83]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlight terraces of the palace. "

text = "<|endoftext|>".join((text1,text2))

print(text)

Hello, do you like tea?<|endoftext|>In the sunlight terraces of the palace. 


In [84]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1131, 988, 1131, 984, 722, 988, 1131, 7]

In [85]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|unk|> the <|unk|> terraces of the <|unk|>.'

In [87]:
import importlib
import tiktoken

print("tiktoken version" , importlib.metadata.version("tiktoken"))

tiktoken version 0.11.0


In [88]:
### This is the Byte pair tokenizer from GPT

tokenizer = tiktoken.get_encoding("gpt2")

In [89]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownplace."   
       )

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 5372, 13]


In [90]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownplace.


In [91]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [92]:
## Take the first 50 tokens for demonstration 
enc_sample = enc_text[50:]

In [93]:
context_size = 4 ## Length of the input

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x:{x}")
print(f"y:{y}")

x:[290, 4920, 2241, 287]
y:[4920, 2241, 287, 257]


In [94]:
for i in range(1, context_size+1): ## Run from 1 to 5
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context , "----->", desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [95]:
for i in range(1, context_size+1): ## Run from 1 to 5
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context) , "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [102]:
### The above is a simple demonstration . For larger data processing we use data loaders
### We will use pytorch's built in Dataset and Dataloader classes

from torch.utils.data import Dataset , DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        #Use the sliding window approach . 
        for i in range(0,len(token_ids) - max_length , stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk)) ## 1'st row i.e X value [from 0 --> 4]
            self.target_ids.append(torch.tensor(target_chunk))  ## 2'nd row i.e Y value [from 1 --> 5]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self,idx):  ## if the idx--> index is 50 (50th row)
        return self.input_ids[idx] , self.target_ids[idx]    ## it will return 50th row of input tensor and 50th row of output tensor

In [103]:
def create_dataloader_v1(txt , batch_size=4, max_length=256 , stride=128 , shuffle=True , drop_last=True , num_workers=0):
    ## txt --> verdict.txt
    ## batch_size --> run 4 process parallelly in cpu
    ## max_length --> context size (previously we used 4 , GPT uses 256)
    ## stride --> How much (words) we need to skip before the next batch
    ## drop_last --> drop the last row , if row size (last batch) is shorter than specified batch size , to prevent loss spikes during training 
    
    ## initialize the tokenizer --> we are using BytePair Tokenizer from GPT
    tokenizer = tiktoken.get_encoding("gpt2")

    ## create Dataloader
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)

    #create Dataloader
    ## checks for the __get_item method in GPTDatasetV1 and returns the item
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [ ]:
### Let's test the dataloader with batch size of 1 for an LLM with context size of 4  

In [109]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
    raw_text = f.read()

In [110]:
import torch
print("pytorch version:" , torch.__version__)
dataloader = create_dataloader_v1(raw_text , batch_size=1 , max_length=4 , stride=1 , shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

pytorch version: 2.8.0+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [111]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [113]:
dataloader = create_dataloader_v1(raw_txt , batch_size=8 , max_length=4 , stride=4 , shuffle=False)
data_iter = iter(dataloader)
inputs , targets = next(data_iter)
print("Inputs : \n :", inputs)
print("\n Targets : \n" , targets)

NameError: name 'raw_txt' is not defined